Author:
        
        PARK, JunHo, junho@ccnets.org

        
        KIM, JeongYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.

In [1]:
import sys

path_append = "../" # Go up one directory from where you are.
sys.path.append(path_append) 

from nn.utils.init import set_random_seed
set_random_seed(0)

import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import torchvision.datasets as dset
from torchvision import transforms

# import albumentations
n_img_sz = 64
# Load the CelebA dataset for training. Specify the root directory where the dataset is located
trainset = dset.CelebA(root=path_append + '../data/celeba', split = "train", transform=transforms.Compose([
                            transforms.Resize(n_img_sz), # Transformations include resizing the images to `n_img_sz`
                            transforms.CenterCrop(n_img_sz), # Center cropping to the same size
                            transforms.ToTensor(), # Converting the images to tensors,
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # Normalizing the pixel values to have a mean and standard deviation of 0.5 across all channels.
                        ]), download= True)

testset = dset.CelebA(root=path_append + '../data/celeba', split = "test", transform=transforms.Compose([
                            transforms.Resize(n_img_sz), # Transformations include resizing the images to `n_img_sz`
                            transforms.CenterCrop(n_img_sz), # Center cropping to the same size
                            transforms.ToTensor(), # Converting the images to tensors
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # Normalizing the pixel values to have a mean and standard deviation of 0.5 across all channels.
                        ]), download= True)    

Files already downloaded and verified
Files already downloaded and verified


In [3]:
label_list = ['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 
              'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 
              'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 
              'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 
              'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young' ]

causal_learning_selected_attributes = torch.tensor([label_list.index('Male')])
causal_learning_none_selected_attributes = torch.tensor([label_list.index('Smiling')])

In [4]:
# Custom dataset class for CelebA dataset
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class CausalModelDataset(Dataset):
    def __init__(self, dataset, selected_attributes):
        self.dataset = dataset
        self.selected_attributes = selected_attributes
    
    def __getitem__(self, index):
        X, y = self.dataset[index]
        y = torch.index_select(y.unsqueeze(0), 1, self.selected_attributes).squeeze(0)
        return X, y
    
    def __len__(self):
        return len(self.dataset)

class EncodingDataset(Dataset):
    def __init__(self, dataset, attributes, causal_model):
        self.dataset = dataset
        self.attributes = attributes
        self.causal_model = causal_model

        data_loader = DataLoader(dataset=dataset, batch_size=256, shuffle=False, drop_last=False)
        list_encodings = []
        list_labels = []
        with torch.no_grad():
            for images, labels in data_loader:
                images = images.to(self.causal_model.device)
                encodings = self.causal_model.explain(images).detach().cpu()
                attributes = labels[:, self.attributes]
                list_encodings.append(encodings)
                list_labels.append(attributes)
        self.encodings = torch.cat(list_encodings, dim=0)
        self.labels = torch.cat(list_labels, dim=0)
        
    def __getitem__(self, index):
        return self.encodings[index], self.labels[index]

    def __len__(self):
        return len(self.dataset)

In [5]:
from tools.setting.ml_params import MLParameters
from tools.setting.data_config import DataConfig
from trainer_hub import TrainerHub
num_classes = 1
data_config = DataConfig(dataset_name = 'celebA', task_type='binary_classification', obs_shape=[3, n_img_sz, n_img_sz], \
                        label_size=num_classes)

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(ccnet_network = 'resnet')

ml_params.training.num_epoch = 1
ml_params.model.ccnet_config.num_layers = 4
ml_params.algorithm.reset_pretrained = True

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print = True)

Trainer Name: causal_trainer


ModelParameters Parameters:


,ccnet_config,ccnet_network,encoder_config,encoder_network
0,See details below,resnet,None,none



Detailed ccnet_config Configuration:


,ccnet_config_model_name,ccnet_config_num_layers,ccnet_config_d_model,ccnet_config_dropout,ccnet_config_obs_shape,ccnet_config_condition_dim,ccnet_config_z_dim
0,resnet,4,256,0.05,"[3, 64, 64]",2,128


TrainingParameters Parameters:


...

,clip_grad_range,decay_rate_100k,learning_rate,max_grad_norm,scheduler_type
0,None,0.05,0.0002,1.0,exponential


AlgorithmParameters Parameters:


,enable_diffusion,error_function,reset_pretrained
0,False,mse,True


DataConfig Parameters:


,dataset_name,task_type,obs_shape,label_size,explain_size,explain_layer,state_size,show_image_indices
0,celeba,binary_classification,"[3, 64, 64]",1,128,tanh,None,None


In [6]:
causal_model_dataset = CausalModelDataset(trainset, causal_learning_selected_attributes)


In [7]:
class AttributeClassifier(torch.nn.Module):
    def __init__(self, input_size, output_size, num_layers=4, hidden_size=128):
        super(AttributeClassifier, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # Create a list to hold all layers
        layers = []
        
        # Input layer
        layers.append(torch.nn.Linear(input_size, hidden_size))
        layers.append(torch.nn.ReLU())
        
        # Hidden layers
        for _ in range(num_layers - 2):
            layers.append(torch.nn.Linear(hidden_size, hidden_size))
            layers.append(torch.nn.ReLU())
        
        # Output layer
        layers.append(torch.nn.Linear(hidden_size, output_size))
        
        # Register all layers
        self.layers = torch.nn.Sequential(*layers)

    def forward(self, x):
        x = self.layers(x)
        return torch.sigmoid(x)

In [8]:
# Function to train classifier
decay_rate = 0.01
iteration_100k = 100000
gamma = pow(decay_rate, 1 / iteration_100k)    
def train_classifier(model, optimizer, trainset):
    model.train()
    
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=gamma)
    train_loader = DataLoader(trainset, batch_size=64, shuffle=True)    
    for data, labels in train_loader:
        data, labels = data.to(device), labels.to(device).float()
        optimizer.zero_grad()
        outputs = model(data)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
    # print learning rate
    print(optimizer.param_groups[0]['lr'])

In [9]:
# Function to evaluate classifier
import torch
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def test_classifier(model, dataset):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
        for data, labels in dataloader:
            data, labels = data.to(device), labels.to(device).float()
            outputs = model(data)
            preds = torch.sigmoid(outputs).round()
            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())
    
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    
    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Accuracy: {accuracy:.4f}')
    print(classification_report(all_labels, all_preds))
    print(confusion_matrix(all_labels, all_preds))
    return accuracy, classification_report(all_labels, all_preds), confusion_matrix(all_labels, all_preds)


In [10]:
# Define a function to store results
def store_results(results_dict, epoch, causal_classifier, none_selected_classifier, testset_with_selected_attributes, testset_with_none_selected_attributes):
    print(f"Testing causal classifier on selected attributes at epoch {epoch}...")
    results_dict['selected'][epoch] = test_classifier(causal_classifier, testset_with_selected_attributes)
    
    print(f"Testing classifier on none selected attributes at epoch {epoch}...")
    results_dict['none_selected'][epoch] = test_classifier(none_selected_classifier, testset_with_none_selected_attributes)

# Create dictionaries to store results
results_dict = {'selected': {}, 'none_selected': {}}

In [11]:
num_epoch = 10
encoding_size = data_config.explain_size
causal_model = trainer_hub.ccnet

selected_classifier = AttributeClassifier(encoding_size, num_classes).to(device)
none_selected_classifier = AttributeClassifier(encoding_size, num_classes).to(device)
selected_optimizer = torch.optim.Adam(selected_classifier.parameters(), lr=0.001)
none_selected_optimizer = torch.optim.Adam(none_selected_classifier.parameters(), lr=0.001)

for epoch in range(num_epoch):
    print(f"Training causal model at epoch {epoch}...")
    trainer_hub.train(causal_model_dataset)
    
    # Train and evaluate classifiers on the explanation datasets
    print("Training causal classifier on selected attributes...")
    trainset_selected_attributes = EncodingDataset(trainset, causal_learning_selected_attributes, causal_model)
    testset_selected_attributes = EncodingDataset(testset, causal_learning_selected_attributes, causal_model)

    train_classifier(selected_classifier, selected_optimizer, trainset_selected_attributes)

    print("Training classifier on none selected attributes...")
    trainset_none_selected_attributes = EncodingDataset(trainset, causal_learning_none_selected_attributes, causal_model)
    testset_none_selected_attributes = EncodingDataset(testset, causal_learning_none_selected_attributes, causal_model)    
    
    train_classifier(none_selected_classifier, none_selected_optimizer, trainset_none_selected_attributes)
    
    # Store and print results
    store_results(results_dict, epoch, selected_classifier, none_selected_classifier, testset_selected_attributes, testset_none_selected_attributes)

Training causal model at epoch 0...


Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

Iterations:   0%|          | 0/2543 [00:00<?, ?it/s]

[0/1][100/2543][Time 13.10]
Unified LR across all optimizers: 0.0001993957766378747
--------------------Training Metrics--------------------
CCNet:  Three Resnet
Inf: 0.1169	Gen: 0.4579	Rec: 0.4546	E: 0.0248	R: 0.0227	P: 0.6780


In [ ]:
# At the end, print all stored results if needed
for key in results_dict:
    print(f"Results for {key}:")
    for epoch, result in results_dict[key].items():
        print(f"Epoch {epoch}:")
        print(result)